    작성일 : 20/03/08
    작성자 : 김은주
    
    작성내용 : 무신사 브랜드 리스트, 랭킹 크롤링_최종프로젝트 모델링 수정 반영
    
    테이블생성 : 
    CREATE TABLE IF NOT EXISTS BRAND_LIST(
    BRD_NO          INT(10) UNSIGNED AUTO_INCREMENT PRIMARY KEY,
    BRD_CODE        VARCHAR(50),
    BRD_NAME_ENG    VARCHAR(50)
    );
    
    CREATE TABLE IF NOT EXISTS BRAND_RANK(
    BRD_CODE        VARCHAR(50),
    BRD_NAME_ENG    VARCHAR(50),
    BRD_RANK        VARCHAR(5),
    LOAD_DATE       TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    );

In [2]:
import requests, pymysql, math, re, time
from bs4 import BeautifulSoup

In [3]:
# ## DB 연결 데이터 
# config = {"host":"192.168.0.12", "user":"musinsa", "passwd":"1111", "db":"PR_TEST"}
# conn = pymysql.connect(**config)

In [4]:
## DB 연결 데이터 
config = {"host":"127.0.0.1", "user":"root", "passwd":"1111", "db":"musinsa_test"}
conn = pymysql.connect(**config)

In [5]:
## BRAND_LIST

def crawl_brand_list() :
    startTime = time.time()
    
    brand_codes = []      #브랜드 코드를 담을 리스트
    brands_eng = []       #브랜드명(영문)을 담을 리스트
    
#     cursor = conn.cursor()
    
    html = ""
    req = requests.get("https://store.musinsa.com/app/contents/brandshop")
    if req.status_code == 200 :
        html = req.text

    soup = BeautifulSoup(html, 'html.parser')
    print("brand_list [HTTP Status : {}]".format(req))

    for li in soup('li'):
        if('class' in li.attrs
            and 'brand_li' in li['class']) :
            brands_eng.append(li.dl.dt.a.get_text().strip())
            brand_codes.append(li.dl.dt.a['href'].replace("/app/brand/goods_list/",""))
            
            
    # DB 적재
    sql = ""
    for i in range(len(brands_eng)) :
        if brands_eng[i] == "I♥NY":
            data = [str(i+1), brand_codes[i], "I LOVE NY"]
            #sql = "INSERT INTO BRAND_LIST(BRD_NAME, BRD_LINK, BRD_CODE) VALUES("+ brands[i] + ", " + brand_links[i] + ", " + brand_codes[i] +")"
            #cursor.execute(sql)
            cursor.execute("insert into BRAND_LIST(BRD_NO, BRD_CODE, BRD_NAME_ENG) values(%s, %s, %s)", data)
            conn.commit()
        else:
            data = [str(i+1), brand_codes[i], brands_eng[i]]
            #sql = "INSERT INTO BRAND_LIST(BRD_NAME, BRD_LINK, BRD_CODE) VALUES("+ brands[i] + ", " + brand_links[i] + ", " + brand_codes[i] +")"
            #cursor.execute(sql)
            cursor.execute("insert into BRAND_LIST(BRD_NO, BRD_CODE, BRD_NAME_ENG) values(%s, %s, %s)", data)
            conn.commit()
            
#     conn.close()
    
    print("--- {} seconds ---".format(time.time() - startTime))

## BRAND_RANK

def crawl_brand_rank():
    startTime = time.time()
    
#     cursor = conn.cursor()
    
    url = "https://store.musinsa.com/app/usr/brand_rank?d_cat_cd=&u_cat_cd=&range=1m&sex=&popup=&page=&sort="

    html = ""
    req = requests.get(url)
    if req.status_code == 200 :
        html = req.text

    soup = BeautifulSoup(html, 'html.parser')
    print("brand_rank [HTTP Status : {}]".format(req))

    brand = []

    for inner in soup.find_all("div", class_="li_inner"):
        rank = inner.find("p", class_="n-label label-default txt_num_rank").text
        brand.append(["brd_rank", str(rank[:rank.index("위")].strip())])
        brand.append(["brd_code", inner.find("p", class_="brand_name").find("a").get("href")[22:].strip()])
        brand.append(["brd_name_eng", inner.find("p", class_="brand_name_en").text.strip()])

        dict_brand = dict(brand)

        ## DB 적재
        cursor = conn.cursor()
        sql = "INSERT INTO BRAND_RANK(BRD_CODE, BRD_NAME_ENG, BRD_RANK) VALUES("\
                     + "'" + dict_brand.get("brd_code", '오류') +"'" +  ","\
                     + "'" + dict_brand.get("brd_name_eng", '오류') +"'" +  ","\
                     + "'" + dict_brand.get("brd_rank", '오류') +"'" + ");"

        cursor.execute(sql)
        conn.commit()
        
#     conn.close()
    print("--- {} seconds ---".format(time.time() - startTime))

In [6]:
# 크롤링 함수 실행
cursor = conn.cursor()

crawl_brand_list()
crawl_brand_rank()

conn.close()

brand_list [HTTP Status : <Response [200]>]
--- 20.25515842437744 seconds ---
brand_rank [HTTP Status : <Response [200]>]
--- 1.5860624313354492 seconds ---
